In [ ]:
from bokeh.io import output_file, output_notebook, show
from bokeh.models import LogColorMapper, WheelZoomTool, ColorBar, LogTicker
from bokeh.plotting import figure
import bokeh.palettes as bp

import numpy as np
import matplotlib.pyplot as plt
from vis_tools import grab_GMT_features, read_HYSPLIT_netCDF

##############################################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '/opt/hysplit/18040212_taupo_3.0_0.01.nc'

# SPECIFY: pick one of 8 time steps (indices 0-7, 3 hr chunks)
TIME_STEP = 0

# SPECIFY: plotting params
ASH_MIN = 10**-2  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
RES = 'f'  # detail of GMT features ('c', 'l', 'i', 'h', 'f')
##############################################################

model = read_HYSPLIT_netCDF(FILENAME)

src_lon = model.attrs['volcano_location'][1]
src_lat = model.attrs['volcano_location'][0]
lon = model['lon'].values
lat = model['lat'].values

time_slice = model['total_deposition'].values[TIME_STEP, :, :]  # grab time step from the gridded data        
np.place(time_slice, time_slice==0, np.nan)  # replace all zero values with nan's

# grab and project features to plot
features = grab_GMT_features(-48, -33, 165, 180, RES)  # this includes all of NZ

p = figure(title='UNPROJECTED', tools='pan, reset')
wz = WheelZoomTool(zoom_on_axis=False)
p.add_tools(wz)
p.toolbar.active_scroll = wz

# PLOT features
p.multi_line(features['longitude'], features['latitude'], color='black')

# PLOT model
x = np.min(lon)
y = np.min(lat)
dw = np.max(lon) - np.min(lon)
dh = np.max(lat) - np.min(lat)
cmapper = LogColorMapper(palette=bp.inferno(256)[::-1], low=ASH_MIN, high=ASH_MAX, nan_color=(0,0,0,0))
p.image(image=[time_slice], x=x, y=y, dw=dw, dh=dh, color_mapper=cmapper, global_alpha=0.5)

# PLOT source location
p.scatter(src_lon, src_lat, size=15, marker='triangle', line_color='black', fill_color='cyan')

p.match_aspect = True
        
p.xaxis.axis_label = 'lon'
p.yaxis.axis_label = 'lat'

p.xgrid.visible = False
p.ygrid.visible = False   

color_bar = ColorBar(color_mapper=cmapper, ticker=LogTicker(), label_standoff=12, border_line_color=None, location=(0,0))
p.add_layout(color_bar, 'right')

output_notebook()    
#output_file('Bokeh_unprojected.html')

show(p)